In [1]:
url = 'https://br.financas.yahoo.com/noticias/acoes-mais-negociadas?offset=0&count=100'

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
response = urlopen(url)

In [4]:
html = response.read()

# Caso de erro na leitura

from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

## vai até network, index.php, request headers, user-agent

url = 'https://br.financas.yahoo.com/noticias/acoes-mais-negociadas'
headers = {}

try: 
    req = Request(url, headers = headers)
    response = urlopen(req)

except HTTPError as e:
    print(e.status, e.reason)
  
except URLError as e:
    print( e.reason)

In [5]:
html = html.decode('utf-8')

In [6]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

html = trata_html(html)

In [7]:
soup = BeautifulSoup(html, features="html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="NoJs desktop" id="atomic" lang="pt-BR">
 <head prefix="og: http://ogp.me/ns#">
  <script>
   window.performance && window.performance.mark && window.performance.mark('PageStart');
  </script>
  <meta charset="utf-8"/>
  <title>
   Painel de Most Actives - Brazil - Yahoo Finanças
  </title>
  <meta content="Painel de ações, setor, associação a índices, compartilhar dados, preço das ações, capitalização de mercado, beta, vendas, lucratividade, índices de avaliação, estimativas dos analistas, valor alto de capital, crescimento da barganha, painéis de ações predefinidos" name="keywords"/>
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <meta content="on" property="twitter:dnt"/>
  <meta content="150897358265131" property="fb:app_id"/>
  <meta content="#400090" name="theme-color"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Veja uma lista de Most Actives - Brazil usando os Painéis do Yahoo Finanças. Crie

In [8]:
alldata = soup.find_all('tbody')
table = alldata[0].find_all('tr')
aux = table[0].find_all('td')
print(aux[0].get_text())
print(aux[1].get_text())
print(aux[2].get_text())
print(aux[3].get_text())
print(aux[4].get_text())
print(aux[5].get_text())
print(aux[6].get_text())
print(aux[7].get_text())

OIBR3.SA
Oi S.A.
1,8000
+0,0400
+2,27%
217,509M
158,093M
10,779B


In [9]:
stock_data = []
a = {'stock': aux[0].get_text(), 'company': aux[1].get_text(), 'price': aux[2].get_text(), 'change': aux[3].get_text(), 
    'perc_change': aux[4].get_text(), 'volume': aux[5].get_text(), 'vol_mean_3m' : aux[6].get_text(),
     'market_cap': aux[7].get_text(), 'PL_index': aux[8].get_text()}
stock_data.append(a)
pd.DataFrame(stock_data)

,stock,company,price,change,perc_change,volume,vol_mean_3m,market_cap,PL_index
0,OIBR3.SA,Oi S.A.,"1,8000","+0,0400","+2,27%","217,509M","158,093M","10,779B",N/A


In [10]:
test = aux[0].find('a')
test

<a class="Fw(600) C($linkColor)" data-reactid="80" href="/quote/OIBR3.SA?p=OIBR3.SA" title="Oi S.A.">OIBR3.SA</a>

In [11]:
test2 = test.get('href')
new_ya = 'https://br.financas.yahoo.com/'
test2.split('?')[0]

new_yan = new_ya + test2.split('?')[0] + '/history?period1'

new_yan

'https://br.financas.yahoo.com//quote/OIBR3.SA/history?period1'

In [12]:
stock_data = []
stock_history = []
new_ya = 'https://br.financas.yahoo.com/'

for stocks in alldata:
    stock = stocks.find_all('tr')
    for infos in stock:
        info = infos.find_all('td')
        a = {'stock': info[0].get_text(), 'company': info[1].get_text(), 'price': info[2].get_text(),
         'change': info[3].get_text(), 'perc_change': info[4].get_text(), 'volume': info[5].get_text(),
         'vol_mean_3m' : info[6].get_text(), 'market_cap': info[7].get_text(), 'PL_index': info[8].get_text()}
        stock_data.append(a)
        site_aux = info[0].find('a')
        site_href = site_aux.get('href')
        new_yan = new_ya + site_href.split('?')[0] + '/history?period1'
        stock_history.append(new_yan)


pd.DataFrame(stock_data).head(20)

,stock,company,price,change,perc_change,volume,vol_mean_3m,market_cap,PL_index
0,OIBR3.SA,Oi S.A.,"1,8000","+0,0400","+2,27%","217,509M","158,093M","10,779B",N/A
1,PETR4.SA,Petróleo Brasileiro S.A. - Petrobras,"22,22","+1,03","+4,86%","127,604M","83,765M","286,958B","40,77"
2,BBDC4.SA,Banco Bradesco S.A.,"23,91","+0,89","+3,87%","72,502M","42,059M","199,925B","12,77"
3,VVAR3.SA,Via Varejo S.A.,"12,11","+0,19","+1,59%","71,644M","44,091M","19,194B","19,25"
4,ITUB4.SA,Itaú Unibanco Holding S.A.,"26,64","+0,81","+3,14%","65,736M","36,844M","247,949B","13,82"
5,COGN3.SA,Cogna Educação S.A.,"3,7000","+0,0300","+0,82%","65,685M","52,033M","6,84B",N/A
6,FNAM11.SA,Fundo de Investimento Amazônia - FINAM,"0,1700","+0,0100","+6,25%","63,355M",N/A,N/A,N/A
7,ITSA4.SA,Itaúsa - Investimentos Itaú SA,"10,21","+0,22","+2,20%","58,055M","29,574M","88,6B","12,17"
8,VALE3.SA,Vale S.A.,"98,86","-1,49","-1,48%","48,492M","30,083M","511,945B","103,84"
9,PETR3.SA,Petróleo Brasileiro S.A. - Petrobras,"21,87","+0,90","+4,29%","47,49M","24,036M","285,401B","40,13"


In [13]:
pd.DataFrame(stock_history).head(5)

,0
0,https://br.financas.yahoo.com//quote/OIBR3.SA/...
1,https://br.financas.yahoo.com//quote/PETR4.SA/...
2,https://br.financas.yahoo.com//quote/BBDC4.SA/...
3,https://br.financas.yahoo.com//quote/VVAR3.SA/...
4,https://br.financas.yahoo.com//quote/ITUB4.SA/...


In [14]:
response2 = urlopen(new_yan)
headdd2 = response2.read()
soup2 = BeautifulSoup(headdd2, features="html.parser")

In [15]:
pre_table = soup2.find('tbody').find_all('tr')
pre_table[0]

<tr class="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)" data-reactid="51"><td class="Py(10px) Ta(start) Pend(10px)" data-reactid="52"><span data-reactid="53">04 de mar de 2021</span></td><td class="Py(10px) Pstart(10px)" data-reactid="54"><span data-reactid="55">18,08</span></td><td class="Py(10px) Pstart(10px)" data-reactid="56"><span data-reactid="57">18,74</span></td><td class="Py(10px) Pstart(10px)" data-reactid="58"><span data-reactid="59">17,97</span></td><td class="Py(10px) Pstart(10px)" data-reactid="60"><span data-reactid="61">18,47</span></td><td class="Py(10px) Pstart(10px)" data-reactid="62"><span data-reactid="63">18,47</span></td><td class="Py(10px) Pstart(10px)" data-reactid="64"><span data-reactid="65">3.541.000</span></td></tr>

In [16]:
a = pre_table[0].find_all('td')

In [17]:
b = {'Data_ref': a[0].get_text(), 'Abrir': a[1].get_text(), 'Alta': a[2].get_text(), 'Baixa': a[3].get_text(), 
    'Fechamento': a[4].get_text(), 'Fechamento_aj': a[5].get_text(), 'Volume' : a[6].get_text() }

In [18]:
b

{'Data_ref': '04 de mar de 2021',
 'Abrir': '18,08',
 'Alta': '18,74',
 'Baixa': '17,97',
 'Fechamento': '18,47',
 'Fechamento_aj': '18,47',
 'Volume': '3.541.000'}

In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [20]:
driver = webdriver.Chrome(executable_path = 'C:\\Program Files\\Chromedriver\\chromedriver.exe')
driver.get("https://www.google.com/")
elem = driver.find_element_by_name("q")
elem.clear()
a = "OIBR"
elem.send_keys(a)
elem.send_keys(Keys.RETURN)

In [21]:
stocks = ['OIBR', 'PETR4']
for stock in stocks:
    driver = webdriver.Chrome(executable_path = 'C:\\Program Files\\Chromedriver\\chromedriver.exe')
    driver.get("https://www.google.com/")
    elem = driver.find_element_by_name("q")
    elem.clear()
    elem.send_keys(stock)
    elem.send_keys(Keys.RETURN)
    driver.quit()

# Testando o Commit